In [2]:
import os
from secret_api_keys import Groq_API_Key

# initialize a chatGroq instance using the provided Groq API key and create an LLM object for interaction

In [3]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key= Groq_API_Key,model_name  = "llama3-groq-70b-8192-tool-use-preview")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000013C97C49A90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000013C97C4A5D0>, model_name='llama3-groq-70b-8192-tool-use-preview', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
llm.invoke("Hii,How are u ?")

AIMessage(content="I'm fine, thank you! How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 17, 'total_tokens': 32, 'completion_time': 0.04546261, 'prompt_time': 0.004319219, 'queue_time': 0.029371556, 'total_time': 0.049781829}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'stop', 'logprobs': None}, id='run-dd42604b-90e8-4a3c-8ce7-c9bcd4a62d21-0', usage_metadata={'input_tokens': 17, 'output_tokens': 15, 'total_tokens': 32})

# Define State of Chatbot


In [5]:
from typing import Annotated # import the Annoted class for type hints with additional metadata
from typing_extensions import TypedDict # import the TypeDict class for defining custom typed Dictionaries

from langgraph.graph import StateGraph # import the StateGraph  class for create State Graph
from langgraph.graph.message import add_messages # import the Add_message function for adding messages to list 


# Define Typed Dictionary to represent the state of the ChatBot

In [6]:
class State(TypedDict):
    message:Annotated[list,add_messages] # List of messages, With add_message function applied

# Create a StateGraph instance , Specifying the state type

In [7]:
graph_Builder = StateGraph(State) 

# Define chatbot function that takes state as input and returns new state with uploaded message
def chatbot(State:State):
    return {"message":[llm.invoke(State["message"])]} # invoke the llm on the input message

# Add the chatbot node to the graph , specfying its name and chatbot function
graph_Builder.add_node("chatbot",chatbot)

# set the chatbot node as both Entry point and finish point
graph_Builder.set_entry_point("chatbot")
graph_Builder.set_finish_point("chatbot")

graph = graph_Builder.compile()



# Stream Graph Based on User Input

In [8]:
def stream_graph_updates(user_input : str):
    #create an initial state with the user input message
    Dict = {"message":[("user",user_input)]} # creting dictionary having one key which is message and it connects to the tuple two elements 'string' and user_input

    #stream updates from the compiled graph
    for event in graph.stream(Dict):
        #loop through each event value (should be only one in this case)
        for value in event.values():
            #print the assistant latest message 
            print("Assitant:",value["message"][-1].content)

In [9]:
while True:
    try:
        user_input = input("user :") # Get input from the user

        if user_input.lower() in ["quit","q","exit"]:
            print("Good Bye!")
            break # Exit the loop

        stream_graph_updates(user_input) # process the input using this graph
    except Exception as e:
        print("Exception is Occured during processing :",e)
        stream_graph_updates(user_input)
        break # exit the loop after the fallback




Assitant: Hi! How can I assist you today?
Assitant: Hello! How can I assist you today?
Assitant: I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have.
Assitant: I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have.
Assitant: I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have.
Good Bye!


In [10]:
def greet(name):
    """Function to greet a user."""
    return f"Hello, {name}!"

print(greet("Ram"))

Hello, Ram!


In [11]:
def display_info(name, age=18, *hobbies):
    """Display user information."""
    print(f"Name: {name}, Age: {age}")
    if hobbies:
        print("Hobbies:", ", ".join(hobbies))

display_info("Alice", 25, "Reading", "Traveling")
display_info("Bob")  # Uses default age

Name: Alice, Age: 25
Hobbies: Reading, Traveling
Name: Bob, Age: 18


In [12]:
def my_function():
    local_var = "I am local"
    return local_var

print(my_function())  # Works fine
# print(local_var)  # This would raise an error since local_var is not defined outside the function.

I am local


In [13]:
def divide(a, b):
    """Divide two numbers and return the result."""
    try:
        return a / b
    except ZeroDivisionError:
        return "Error! Division by zero."

print(divide(10, 2))  # Output: 5.0
print(divide(10, 0))  # Output: Error! Division by zero

5.0
Error! Division by zero.


In [14]:
def area_circle(radius):
    """Calculate area of a circle."""
    import math
    return math.pi * radius ** 2

def area_rectangle(length, width):
    """Calculate area of a rectangle."""
    return length * width

print(f"Area of circle: {area_circle(5)}")
print(f"Area of rectangle: {area_rectangle(4, 6)}")  # Output: Area of rectangle: 24

Area of circle: 78.53981633974483
Area of rectangle: 24
